# ETL 02 - Scripts Area Stage SQL
    Objetivo: Popular as tabelas no banco DSTG_OiMasterDados
    Desenvolvedor: Marcelo Muniz de Alencar

In [ ]:
# importar pacote pandas
import pandas as pd 

# importar sqlalchemy, urllib, pyodbc
from sqlalchemy import create_engine
import urllib
import pyodbc

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# criando o dataframe a partir do csv
df = pd.read_csv("C:/city_temperature.csv") 

# visualizando campos dataframe
df.head()

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


In [ ]:
# Carregando a tabela TOiSTG_CityTemperatureCsv
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=PCMARCELO\SQLEXPRESS;DATABASE=DSTG_OiMasterDados") # alterar linha para seu servidor
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

try:
    df.to_sql('TOiSTG_CityTemperatureCsv', schema='OiMasterDados', con = engine, chunksize=100, method='multi', index=False, if_exists='append')
    print('Executado o script SQL para carga da tabela TOiSTG_CityTemperatureCsv no banco DSTG_OiMasterDados.')
except:
    print('Erro. Contactar desenvolvedor!')

Executado o script SQL para carga da tabela TOiSTG_CityTemperatureCsv no banco DSTG_OiMasterDados.


In [ ]:
# conexão banco de dados 'DSTG_OiMasterDados' no servidor
connDSTG = pyodbc.connect('Driver={SQL Server};'
                     'Server=PCMARCELO\SQLEXPRESS;' # alterar linha para seu servidor
                     'Database=DSTG_OiMasterDados;'
                     'Trusted_Connection=yes;')

# Carregando a tabela TOiSTG_Cidade  
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=PCMARCELO\SQLEXPRESS;DATABASE=DSTG_OiMasterDados") # alterar linha para seu servidor
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

cursorDSTG = connDSTG.cursor()

sqlDSTG_TOiSTG_Cidade_Truncate = ''' 
use DSTG_OiMasterDados
TRUNCATE TABLE DSTG_OiMasterDados.OiMasterDados.TOiSTG_Cidade
'''

sqlDSTG_TOiSTG_Cidade = ''' 
              SELECT    ROW_NUMBER() OVER (ORDER BY TT.RegiaoGlobo,TT.Pais,TT.Estado,TT.Cidade) as NumCidade,
                        TT.Cidade,
                        TT.Estado,
                        TT.Pais,
                        TT.RegiaoGlobo
              FROM
              (SELECT DISTINCT 
                     [Region]      as RegiaoGlobo
                    ,[Country]     as Pais
                    ,[State]       as Estado
                    ,[City]        as Cidade
              FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_CityTemperatureCsv] (NOLOCK)) AS TT
              ORDER BY 5,4,3,2 '''
dfDSTG_TOiSTG_Cidade = pd.read_sql(sqlDSTG_TOiSTG_Cidade, connDSTG)

try:
    cursorDSTG.execute(sqlDSTG_TOiSTG_Cidade_Truncate)
    connDSTG.commit()
    
    dfDSTG_TOiSTG_Cidade.to_sql('TOiSTG_Cidade', schema='OiMasterDados', con = engine, chunksize=100, method='multi', index=False, if_exists='append')
    print('Executado o script SQL para carga da tabela TOiSTG_Cidade no banco DSTG_OiMasterDados.')
except:
    print('Erro. Contactar desenvolvedor!')

Executado o script SQL para carga da tabela TOiSTG_Cidade no banco DSTG_OiMasterDados.


In [ ]:
# conexão banco de dados 'DSTG_OiMasterDados' no servidor
connDSTG = pyodbc.connect('Driver={SQL Server};'
                     'Server=PCMARCELO\SQLEXPRESS;' # alterar linha para seu servidor
                     'Database=DSTG_OiMasterDados;'
                     'Trusted_Connection=yes;')

# Carregando a tabela TOiSTG_Calendario  
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=PCMARCELO\SQLEXPRESS;DATABASE=DSTG_OiMasterDados")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

cursorDSTG = connDSTG.cursor()

sqlDSTG_TOiSTG_Calendario_Truncate = ''' 
use DSTG_OiMasterDados
TRUNCATE TABLE DSTG_OiMasterDados.OiMasterDados.TOiSTG_Calendario
'''

sqlDSTG_TOiSTG_Calendario = ''' 
DECLARE @Year INT = 1900;
DECLARE @YearCnt INT = (year(getdate()) - @Year)+1 ;
DECLARE @StartDate DATE = DATEFROMPARTS(@Year, '01','01')
DECLARE @EndDate DATE = DATEADD(DAY, -1, DATEADD(YEAR, @YearCnt, @StartDate));

;WITH Cal(n) AS
(
SELECT 0 UNION ALL SELECT n + 1 FROM Cal
WHERE n < DATEDIFF(DAY, @StartDate, @EndDate)
),
FnlDt(d) AS
(
SELECT DATEADD(DAY, n, @StartDate) FROM Cal
),
FinalCte AS
(
SELECT
replace(d,'-','') as NumData,
CONVERT(DATE,d) as DescData,
DATEPART(YEAR, d) as NumAno,
'Ano ' + convert(varchar(10),DATEPART(YEAR, d)) as DescAno,
case when MONTH(d) IN (1,2,3,4,5,6)
		then 1
		else 2
end as NumSemestre,
case when MONTH(d) IN (1,2,3,4,5,6)
		then '1º Semestre'
		else '2º Semestre'
end as DescSemestre,
case when MONTH(d) IN (1,2,3)
		then 1
	when MONTH(d) IN (4,5,6)
		then 2
	when MONTH(d) IN (7,8,9)
		then 3
	when MONTH(d) IN (10,11,12)
		then 4
end as NumTrimestre,
case when MONTH(d) IN (1,2,3)
		then '1º Trimestre'
	when MONTH(d) IN (4,5,6)
		then '2º Trimestre'
	when MONTH(d) IN (7,8,9)
		then '3º Trimestre'
	when MONTH(d) IN (10,11,12)
		then '4º Trimestre'
end as DescTrimestre,
MONTH(d) as NumMes, 
DATENAME(MONTH, d) as DescMes,
datepart(WEEK,d) as NumSemanaAno,
CONVERT(varchar(10),datepart(WEEK,d)) + 'ª Semana' as DescSemanaAno,
DATEPART(DAY, d) as NumDia,
datepart(WEEKDAY,d) as NumDiaSemana,
DATENAME(WEEKDAY, d) as DescDiaSemana,
case when datepart(WEEKDAY,d) IN (1,7)
		then 'S'
		else 'N'
end as FlgFinalSemana

FROM FnlDt
)
SELECT * FROM finalCte
ORDER BY NumData 
OPTION (MAXRECURSION 0); '''

dfDSTG_TOiSTG_Calendario = pd.read_sql(sqlDSTG_TOiSTG_Calendario, connDSTG)

try:
    cursorDSTG.execute(sqlDSTG_TOiSTG_Calendario_Truncate)
    connDSTG.commit()
    
    dfDSTG_TOiSTG_Calendario.to_sql('TOiSTG_Calendario', schema='OiMasterDados', con = engine, chunksize=100, method='multi', index=False, if_exists='append')
    print('Executado o script SQL para carga da tabela TOiSTG_Calendario no banco DSTG_OiMasterDados.')
except:
    print('Erro. Contactar desenvolvedor!')

Executado o script SQL para carga da tabela TOiSTG_Cidade no banco DSTG_OiMasterDados.


In [ ]:
# conexão banco de dados 'DSTG_OiMasterDados' no servidor
connDSTG = pyodbc.connect('Driver={SQL Server};'
                     'Server=PCMARCELO\SQLEXPRESS;' # alterar linha para seu servidor
                     'Database=DSTG_OiMasterDados;'
                     'Trusted_Connection=yes;')

# Carregando a tabela TOiSTG_FatoTemperatura  
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=PCMARCELO\SQLEXPRESS;DATABASE=DSTG_OiMasterDados")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

cursorDSTG = connDSTG.cursor()

sqlDSTG_TOiSTG_FatoTemperatura_Truncate = ''' 
use DSTG_OiMasterDados
TRUNCATE TABLE DSTG_OiMasterDados.OiMasterDados.TOiSTG_FatoTemperatura
'''

sqlDSTG_TOiSTG_FatoTemperatura = ''' 
SELECT 
	  convert(int,isnull(t1.NumData,'19000101'))	as NumData,
	  CONVERT(INT,convert(varchar(10),t2.[Year]) + 
                 REPLICATE('0', 2 - LEN(t2.[Month])) + RTrim(t2.[Month]) + 
				 REPLICATE('0', 2 - LEN(t2.[Day])) + RTrim(t2.[Day]))
							as NumDataOrigem,
	  case when len((CONVERT(varchar(20), t1.NumData))) <> 8
			then 'N'
			else 'S'
	  end								as FlgFormDataCorreta,
	  convert(int,t3.NumCidade)			as NumCidade,
	  round((T2.AvgTemperature-32)/1.8,2)
							as ValMedioTemperaturaC
FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_CityTemperatureCsv] AS T2 (NOLOCK)
			left outer join
	   [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_Calendario] as t1 (nolock)
		on CONVERT(INT,t1.NumData) = CONVERT(INT,convert(varchar(10),t2.[Year]) + 
                 REPLICATE('0', 2 - LEN(t2.[Month])) + RTrim(t2.[Month]) + 
				 REPLICATE('0', 2 - LEN(t2.[Day])) + RTrim(t2.[Day]))
			left outer JOIN
	 [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_Cidade] AS T3 (NOLOCK)
		ON  RTRIM(LTRIM(T2.[City])) = RTRIM(LTRIM(T3.Cidade))
		and RTRIM(LTRIM(T2.Country)) = RTRIM(LTRIM(T3.Pais))
		and RTRIM(LTRIM(T2.Region)) = RTRIM(LTRIM(T3.RegiaoGlobo))
		and RTRIM(LTRIM(isnull(T2.[State],''))) = RTRIM(LTRIM(isnull(T3.Estado,'')));
'''

dfDSTG_TOiSTG_FatoTemperatura = pd.read_sql(sqlDSTG_TOiSTG_FatoTemperatura, connDSTG)

try:
    cursorDSTG.execute(sqlDSTG_TOiSTG_FatoTemperatura_Truncate)
    connDSTG.commit()
    
    dfDSTG_TOiSTG_FatoTemperatura.to_sql('TOiSTG_FatoTemperatura', schema='OiMasterDados', con = engine, chunksize=100, method='multi', index=False, if_exists='append')
    print('Executado o script SQL para carga da tabela TOiSTG_FatoTemperatura no banco DSTG_OiMasterDados.')
except:
    print('Erro. Contactar desenvolvedor!')

Executado o script SQL para carga da tabela TOiSTG_FatoTemperatura no banco DSTG_OiMasterDados.


## FIM